[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/05_Devices_and_HumanInLoop.ipynb)


In [ ]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


# Sensor/Pump Models and Human-in-the-loop

Simulate sensor noise, pump caps, and real-time interventions.


In [ ]:
import json
from pathlib import Path

import iints
import yaml
from iints.core.algorithms.mock_algorithms import ConstantDoseAlgorithm
from iints.core.devices import SensorModel, PumpModel
from iints.core.simulator import Simulator
from iints.core.patient.models import PatientModel
from iints.validation import build_stress_events

presets = json.loads(Path("src/iints/presets/presets.json").read_text())
preset = next(p for p in presets if p["name"] == "baseline_t1d")

patient_config = yaml.safe_load(Path("src/iints/data/virtual_patients/{}".format(preset["patient_config"] + ".yaml")).read_text())
patient_model = PatientModel(**iints.validation.validate_patient_config_dict(patient_config).model_dump())

sensor = SensorModel(noise_std=1.0, bias=2.0, lag_minutes=0, dropout_prob=0.0, seed=1)
pump = PumpModel(max_units_per_step=0.5, quantization_units=0.05, dropout_prob=0.01, seed=1)

def on_step(ctx):
    # If glucose is too low, inject 12g rescue carbs
    if ctx["glucose_actual_mgdl"] < 70:
        return {"additional_carbs": 12, "note": "rescue carbs"}
    return None

simulator = Simulator(
    patient_model=patient_model,
    algorithm=ConstantDoseAlgorithm(dose=0.2),
    time_step=preset["time_step_minutes"],
    sensor_model=sensor,
    pump_model=pump,
    on_step=on_step,
    seed=42,
)

for event in build_stress_events(preset["scenario"]["stress_events"]):
    simulator.add_stress_event(event)

results, safety = simulator.run_batch(duration_minutes=240)
results[["time_minutes", "glucose_actual_mgdl", "sensor_status", "pump_status", "human_intervention"]].head()
